# 2강. 파이썬 응용 및 크롤링

데이터를 분석하기에 앞서 직접 데이터를 수집합니다. AI 모델링의 시작은 데이터에 대한 이해를 높이는 것입니다. 최근 Data-driven Modeling 방식이 대두되고 있습니다. 모델일 상향 평준화된 만큼, 모델 구조를 바꾸기보다는 어떤 데이터를 입력해야 성능을 높일 수 있는지 연구하는 것입니다. 아무리 모델이 좋더라도 학습에 활용되는 데이터의 퀄리티가 높지 않으면 성능이 떨어질 수밖에 없습니다. 또한, 모델링에 집중하다 보면 내가 처리하고자 하는 데이터에 대한 이해를 놓치는 경우가 많습니다. 비록 앞으로 직접 데이터를 수집하는 일이 많지 않다고 하더라도, 데이터를 먼저 살펴보시기를 권장합니다.

## Selenium 활용 동적 크롤링

Selenium은 웹을 동작시키는 하나의 도구입니다. Colab 환경에서는 실제로 웹이 동작하는 화면을 띄울 수는 없지만, 가상의 브라우저를 동작시킬 수 있습니다.

Selenium 패키지는 자주 업데이트 되기 때문에 Colab 환경에서 Selenium 설치하는 방법은 계속 매번 달라집니다. 여기서 코드는 [블로그](https://velog.io/@kite_day/colab-%EC%97%90%EC%84%9C-%EC%9B%B9-%ED%81%AC%EB%A1%A4%EB%A7%81%ED%95%98%EA%B8%B0-selenium)를 참고했습니다.

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver '/content/drive/MyDrive/Colab Notebooks'
!pip install chromedriver-autoinstaller

zsh:1: command not found: apt-get
The operation couldn’t be completed. Unable to locate a Java Runtime.
Please visit http://www.java.com for information on installing Java.

cp: /usr/lib/chromium-browser/chromedriver: No such file or directory


In [3]:
# selenium 설치 확인
!python --version

import selenium
print(selenium.__version__)

Python 3.12.2
4.25.0


In [4]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import sys
from selenium.webdriver.common.keys import Keys
import urllib.request
import os
from urllib.request import urlretrieve

import time
import pandas as pd
import chromedriver_autoinstaller  # setup chrome options

In [5]:
chrome_path = "/content/drive/MyDrive/Colab Notebooks/chromedriver"

In [6]:
sys.path.insert(0,chrome_path)
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless') # Colab은 새창을 지원하지않기 때문에 창을 띄우지 않는 Headless 모드로 실행해야 합니다.
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')  # set path to chromedriver as per your configuration
chrome_options.add_argument('lang=ko_KR') # 한국어

chromedriver_autoinstaller.install()  # set the target URL

'/opt/anaconda3/lib/python3.12/site-packages/chromedriver_autoinstaller/129/chromedriver'

## 스포츠 네이버 뉴스 기사 데이터 수집

Selenium과 BeautifulSoup을 활용해서 네이버에서 제공하는 스포츠 뉴스 기사를 수집합니다.

In [27]:
driver = webdriver.Chrome(options=chrome_options)

In [28]:
# 네이버에서 제공하는 링크
url = 'https://sports.news.naver.com/wfootball/news/index?isphoto=N&date=20240927'

# 드라이버로 URL 접속하기
driver.get(url)

In [29]:
# 드라이버에서 페이지 소스 코드 불러오기
html = driver.page_source

In [30]:
# BeautifulSoup 객체 생성
from bs4 import BeautifulSoup

soup = BeautifulSoup(html, 'html.parser')

In [31]:
# 정규표현식 라이브러리
import re

# BeautifulSoup으로 뉴스 리스트 영역 설정
news_section = soup.find('div', re.compile('news_list'))

# 영역 내에서 뉴스 리스트 불러오기
news_tag_list = news_section.find_all('li')
news_tag_list

[<li>
 <a class="thmb" href="https://m.sports.naver.com/wfootball/article/413/0000184297" onclick="clickcr(this, 'nwl.image', '', '', event);">
 <img alt="'맨유 팬들 서운하다' 데뷔골 터뜨린 맥토미니, &quot;나폴리 팬들이 세계 최고&quot;" class="lazyLoadImage" lazy-src="https://imgnews.pstatic.net/image/origin/413/2024/09/27/184297.jpg?type=w140" src="https://imgnews.pstatic.net/image/origin/413/2024/09/27/184297.jpg?type=w140" width="140"/>
 <span class="mask"></span>
 </a>
 <div class="text">
 <a class="title" href="https://m.sports.naver.com/wfootball/article/413/0000184297" onclick="clickcr(this, 'nwl.title', '', '', event);"><span>'맨유 팬들 서운하다' 데뷔골 터뜨린 맥토미니, "나폴리 팬들이 세계 최고"</span></a>
 <p class="desc">사진 = BBC스콧 맥토미니가 나폴리에 애정을 드러냈다.나폴리는 27일 오전 4시(이하 한국시간) 이탈리아 나폴리에 위치한 스타디오 디에고 아르만도 마라도나...</p>
 <div class="source">
 <span class="press">인터풋볼</span>
 <span class="time"><span class="bar"></span>2024.09.27 23:50</span>
 </div>
 </div>
 </li>,
 <li>
 <a class="thmb" href="https://m.sports.naver.com/wfootball/articl

In [32]:
# Map 함수를 활용해 뉴스 제목 리스트 만들기
list(map(lambda x : x.find('a', re.compile('title')).text, news_tag_list))

['\'맨유 팬들 서운하다\' 데뷔골 터뜨린 맥토미니, "나폴리 팬들이 세계 최고"',
 '\'선발 유력\' 이강인의 달라진 위상, "PSG 대표하는 스타될 수 있어" 현지 매체의 \'극찬\'',
 "17살에 주급 5300만원…'18G 32골' U-18 PL 득점왕, 아스널 NO 맨유행 'Here we go'",
 '그동안 후보였던 이유가 있었다! 전반 7분 만에 어이없는 반칙 → 다이렉트 퇴장...英 매체 "끔찍한 결정이었다" 혹평',
 '로드리 \'시즌 OUT\' 확정!…과르디올라 "십자인대+반월판 수술 진행, 다음 시즌에 돌아올 것" [오피셜]',
 '"비참하게 실패한" 첼시 \'1000억\' 사나이, 탈출 원한다... 계약 7년 남았는데?',
 '\'히로키에 밀린다고?\' 김민재, 증명의 기회 왔다…"레버쿠젠 상대로 안정된 실력 증명할 것"',
 '손흥민 천만다행, 허벅지 붙잡았어도 부상 피했다…"훈련 참여 원해, 심하지 않은 듯"',
 "리그 우승 도전에 치명적...'승리 요정' 로드리, 시즌아웃...전방십자인대-반월판 수술 [공식발표]",
 '"너희들은 그래서 절대 우승하지 못해!"…\'독설가\'의 비판, "아스널, 작은 클럽의 작은 사고방식"',
 '"재계약? 대화 없었다" 손흥민→토트넘 압박한 포스텍 "SON 계속 머물기를 바란다"',
 '"김민재? 패스 안 돼! 볼 간수도 떨어지는 수비수"…키커가 이런 폭언까지',
 "바르셀로나, 결국 결단 내렸다! '슬개건 부상' 주전 GK 장기 후계자로 포르투갈 국대 GK 낙점...내년 여름 영입 시도",
 '\'맨시티 초비상\' 로드리 시즌 아웃 우려가 현실로…펩 감독 "로드리의 이번 시즌은 끝났어"',
 '토트넘, \'부상\' 손흥민 상황 업데이트..."훈련 참여하고자 해, 맨유 원정 전까지 계속 확인할 것"',
 '솔샤르 맨유 복귀? 웃기는 소리! "텐 하흐, 전폭적 지지 받고 있어" 로마노 컨펌',
 '드디어! 이강인 PSG 주전 올라서나? 렌전도 선발 출격 전망→이번엔 중원 아닌 ‘왼쪽 윙어’ 기용

In [13]:
# Map 함수를 활용해 뉴스 속성 리스트 만들기 - 링크들만 나열
# 크롤링 시 해당되는 링크들을 나열해 맵으로 묶어 사용하는 경우가 많다
list(map(lambda x : x.find('a', re.compile('thmb')).get('href'), news_tag_list))


['https://m.sports.naver.com/wfootball/article/413/0000184297',
 'https://m.sports.naver.com/wfootball/article/411/0000052771',
 'https://m.sports.naver.com/wfootball/article/311/0001776375',
 'https://m.sports.naver.com/wfootball/article/139/0002210965',
 'https://m.sports.naver.com/wfootball/article/311/0001776373',
 'https://m.sports.naver.com/wfootball/article/413/0000184296',
 'https://m.sports.naver.com/wfootball/article/411/0000052770',
 'https://m.sports.naver.com/wfootball/article/477/0000514052',
 'https://m.sports.naver.com/wfootball/article/109/0005164238',
 'https://m.sports.naver.com/wfootball/article/117/0003874891',
 'https://m.sports.naver.com/wfootball/article/413/0000184293',
 'https://m.sports.naver.com/wfootball/article/311/0001776365',
 'https://m.sports.naver.com/wfootball/article/139/0002210962',
 'https://m.sports.naver.com/wfootball/article/411/0000052769',
 'https://m.sports.naver.com/wfootball/article/109/0005164215',
 'https://m.sports.naver.com/wfootball/a

In [33]:
# Map 함수를 활용해 중복되는 부분을 제외한 뉴스 링크 만들기
list(map(lambda x : x.find('a', re.compile('thmb')).get('href')[len('https://m.sports.naver.com'):], news_tag_list))

['/wfootball/article/413/0000184297',
 '/wfootball/article/411/0000052771',
 '/wfootball/article/311/0001776375',
 '/wfootball/article/139/0002210965',
 '/wfootball/article/311/0001776373',
 '/wfootball/article/413/0000184296',
 '/wfootball/article/411/0000052770',
 '/wfootball/article/477/0000514052',
 '/wfootball/article/109/0005164238',
 '/wfootball/article/117/0003874891',
 '/wfootball/article/413/0000184293',
 '/wfootball/article/311/0001776365',
 '/wfootball/article/139/0002210962',
 '/wfootball/article/411/0000052769',
 '/wfootball/article/109/0005164215',
 '/wfootball/article/413/0000184292',
 '/wfootball/article/144/0000991205',
 '/wfootball/article/411/0000052768',
 '/wfootball/article/477/0000514048',
 '/wfootball/article/413/0000184291']

In [15]:
# Selenium의 기능을 활용해 Xpath로 요소를 찾고, 버튼 클릭하기

epl_button = driver.find_element(By.XPATH, '//*[@id="_sectionList"]/li[2]/a/span')
epl_button.click()

In [16]:
# Python 응용: 반복문을 활용해 버튼 클릭하기
import time

n = 0
for i in range(10):
    epl_button.click()
    
    time.sleep(2) # 웹을 동작시킨 후, 정보를 불러올 수 있도록 시간을 기다려야 한다. 
    n += 1
    print(f'{n}회 클릭했습니다.')

1회 클릭했습니다.
2회 클릭했습니다.
3회 클릭했습니다.
4회 클릭했습니다.
5회 클릭했습니다.
6회 클릭했습니다.
7회 클릭했습니다.
8회 클릭했습니다.
9회 클릭했습니다.
10회 클릭했습니다.


In [34]:
# 다시 소스 코드 불러오기
html = driver.page_source # 웹을 동작시킨 뒤에는 다시 소스 코드를 불러와야 업데이트된 정보를 반영할 수 있습니다.
soup = BeautifulSoup(html, 'html.parser')

news_section = soup.find('div', re.compile('news_list'))
news_tag_list = news_section.find_all('li')

news_title_list = list(map(lambda x: x.find('a', re.compile('title')).get_text(), news_tag_list))
news_link_list = list(map(lambda x: x.find('a', re.compile('thmb')).get('href'), news_tag_list))

In [35]:
# 수집된 뉴스 개수 확인
print(f'뉴스 제목 수: {len(news_title_list)}, 뉴스 링크 수: {len(news_link_list)}')

뉴스 제목 수: 20, 뉴스 링크 수: 20


In [36]:
# Zip 함수를 활용해 뉴스 리스트 만들기
news_list = list(zip(news_title_list, news_link_list)); news_list

[('\'맨유 팬들 서운하다\' 데뷔골 터뜨린 맥토미니, "나폴리 팬들이 세계 최고"',
  'https://m.sports.naver.com/wfootball/article/413/0000184297'),
 ('\'선발 유력\' 이강인의 달라진 위상, "PSG 대표하는 스타될 수 있어" 현지 매체의 \'극찬\'',
  'https://m.sports.naver.com/wfootball/article/411/0000052771'),
 ("17살에 주급 5300만원…'18G 32골' U-18 PL 득점왕, 아스널 NO 맨유행 'Here we go'",
  'https://m.sports.naver.com/wfootball/article/311/0001776375'),
 ('그동안 후보였던 이유가 있었다! 전반 7분 만에 어이없는 반칙 → 다이렉트 퇴장...英 매체 "끔찍한 결정이었다" 혹평',
  'https://m.sports.naver.com/wfootball/article/139/0002210965'),
 ('로드리 \'시즌 OUT\' 확정!…과르디올라 "십자인대+반월판 수술 진행, 다음 시즌에 돌아올 것" [오피셜]',
  'https://m.sports.naver.com/wfootball/article/311/0001776373'),
 ('"비참하게 실패한" 첼시 \'1000억\' 사나이, 탈출 원한다... 계약 7년 남았는데?',
  'https://m.sports.naver.com/wfootball/article/413/0000184296'),
 ('\'히로키에 밀린다고?\' 김민재, 증명의 기회 왔다…"레버쿠젠 상대로 안정된 실력 증명할 것"',
  'https://m.sports.naver.com/wfootball/article/411/0000052770'),
 ('손흥민 천만다행, 허벅지 붙잡았어도 부상 피했다…"훈련 참여 원해, 심하지 않은 듯"',
  'https://m.sports.naver.com/wfootball/article/477/

In [37]:
# 사용자 정의 함수를 활용해 튜플 형태로 데이터 불러오기

def get_news(news_tag):
    news_title = news_tag.find('span', re.compile('NewsList_title')).text
    news_link = news_tag.find('a', re.compile('NewsList_link')).get('href')
    
    return news_title, news_link

In [38]:
# Pandas를 활용해 데이터 프레임 생성하기
import pandas as pd

df = pd.DataFrame(news_list, columns=['title', 'url'])
df

,title,url
0,"'맨유 팬들 서운하다' 데뷔골 터뜨린 맥토미니, ""나폴리 팬들이 세계 최고""",https://m.sports.naver.com/wfootball/article/4...
1,"'선발 유력' 이강인의 달라진 위상, ""PSG 대표하는 스타될 수 있어"" 현지 매체...",https://m.sports.naver.com/wfootball/article/4...
2,"17살에 주급 5300만원…'18G 32골' U-18 PL 득점왕, 아스널 NO 맨...",https://m.sports.naver.com/wfootball/article/3...
3,그동안 후보였던 이유가 있었다! 전반 7분 만에 어이없는 반칙 → 다이렉트 퇴장.....,https://m.sports.naver.com/wfootball/article/1...
4,"로드리 '시즌 OUT' 확정!…과르디올라 ""십자인대+반월판 수술 진행, 다음 시즌에...",https://m.sports.naver.com/wfootball/article/3...
5,"""비참하게 실패한"" 첼시 '1000억' 사나이, 탈출 원한다... 계약 7년 남았는데?",https://m.sports.naver.com/wfootball/article/4...
6,"'히로키에 밀린다고?' 김민재, 증명의 기회 왔다…""레버쿠젠 상대로 안정된 실력 증...",https://m.sports.naver.com/wfootball/article/4...
7,"손흥민 천만다행, 허벅지 붙잡았어도 부상 피했다…""훈련 참여 원해, 심하지 않은 듯""",https://m.sports.naver.com/wfootball/article/4...
8,"리그 우승 도전에 치명적...'승리 요정' 로드리, 시즌아웃...전방십자인대-반월판...",https://m.sports.naver.com/wfootball/article/1...
9,"""너희들은 그래서 절대 우승하지 못해!""…'독설가'의 비판, ""아스널, 작은 클럽의...",https://m.sports.naver.com/wfootball/article/1...


In [52]:
# Pandas의 내장 함수를 활용해 날짜 범위 계산하기
date_range = pd.date_range('2024-09-20', '2024-09-28').strftime('%Y%m%d')
date_range

Index(['20240920', '20240921', '20240922', '20240923', '20240924', '20240925',
       '20240926', '20240927', '20240928'],
      dtype='object')

In [54]:
# 반복문을 활용해 날짜별로 데이터 수집하기
import pandas as pd
from tqdm.auto import tqdm

total_news_list = []
for date in tqdm(date_range):
    url = f'https://sports.news.naver.com/wfootball/news/index?isphoto=N&date={date}'
    driver.get(url)
    time.sleep(2)

    news_more_button = driver.find_element(By.XPATH, '//*[@id="_sectionList"]/li[2]/a/span')

    n = 0
    for i in range(5):
        try:
            news_more_button.click()
            time.sleep(2)
            n += 1
            print(f'{date}: {n}회 클릭했습니다.')
        except:
            next

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    


    news_section = soup.find('div', re.compile('news_list'))
    news_tag_list = news_section.find_all('li')

    news_title_list = list(map(lambda x: x.find('a', re.compile('title')).get_text(), news_tag_list))
    news_link_list = list(map(lambda x: x.find('a', re.compile('thmb')).get('href'), news_tag_list))
    
    date_list = [date] * len(news_link_list)

    daily_news_list = list(zip(news_title_list, news_link_list, date_list))
    total_news_list.extend(daily_news_list)

  0%|          | 0/9 [00:00<?, ?it/s]

20240920: 1회 클릭했습니다.
20240920: 2회 클릭했습니다.
20240920: 3회 클릭했습니다.
20240920: 4회 클릭했습니다.
20240920: 5회 클릭했습니다.
20240921: 1회 클릭했습니다.
20240921: 2회 클릭했습니다.
20240921: 3회 클릭했습니다.
20240921: 4회 클릭했습니다.
20240921: 5회 클릭했습니다.
20240922: 1회 클릭했습니다.
20240922: 2회 클릭했습니다.
20240922: 3회 클릭했습니다.
20240922: 4회 클릭했습니다.
20240922: 5회 클릭했습니다.
20240923: 1회 클릭했습니다.
20240923: 2회 클릭했습니다.
20240923: 3회 클릭했습니다.
20240923: 4회 클릭했습니다.
20240923: 5회 클릭했습니다.
20240924: 1회 클릭했습니다.
20240924: 2회 클릭했습니다.
20240924: 3회 클릭했습니다.
20240924: 4회 클릭했습니다.
20240924: 5회 클릭했습니다.
20240925: 1회 클릭했습니다.
20240925: 2회 클릭했습니다.
20240925: 3회 클릭했습니다.
20240925: 4회 클릭했습니다.
20240925: 5회 클릭했습니다.
20240926: 1회 클릭했습니다.
20240926: 2회 클릭했습니다.
20240926: 3회 클릭했습니다.
20240926: 4회 클릭했습니다.
20240926: 5회 클릭했습니다.
20240927: 1회 클릭했습니다.
20240927: 2회 클릭했습니다.
20240927: 3회 클릭했습니다.
20240927: 4회 클릭했습니다.
20240927: 5회 클릭했습니다.
20240928: 1회 클릭했습니다.
20240928: 2회 클릭했습니다.
20240928: 3회 클릭했습니다.
20240928: 4회 클릭했습니다.
20240928: 5회 클릭했습니다.


In [55]:
# 수집한 데이터로 최종 데이터 프레임 생성하기
df = pd.DataFrame(total_news_list, columns=['title', 'url', 'date'])
df

,title,url,date
0,"손흥민 EPL 베스트11 선정, 토트넘 대표 선수로 뽑혔다... 홀란·살라 미친 스리톱",https://m.sports.naver.com/wfootball/article/1...,20240920
1,"""김민재, 콤파니 감독과 호흡 좋다"" 평가에도 '키커'는 여전히 부정적...""이토가...",https://m.sports.naver.com/wfootball/article/1...,20240920
2,"백승호, 3부 탈출 꿈도 못 꾸나... 감독 사랑 독차지 ""앞으로 몇 년 동안 팀 핵심""",https://m.sports.naver.com/wfootball/article/4...,20240920
3,"당당했던 태도는 어디에? 태세 전환한 슬롯, ""리버풀에 에이스 3인방 있어 너무 행복해""",https://m.sports.naver.com/wfootball/article/4...,20240920
4,"'결국 탈났다' 강행군 소화하던 손흥민, 경기 도중 부상 의심 교체 아웃...A매치...",https://m.sports.naver.com/wfootball/article/4...,20240920
...,...,...,...
58,"백승호, 3부 탈출 꿈도 못 꾸나... 감독 사랑 독차지 ""앞으로 몇 년 동안 팀 핵심""",https://m.sports.naver.com/wfootball/article/4...,20240928
59,"당당했던 태도는 어디에? 태세 전환한 슬롯, ""리버풀에 에이스 3인방 있어 너무 행복해""",https://m.sports.naver.com/wfootball/article/4...,20240928
60,"'결국 탈났다' 강행군 소화하던 손흥민, 경기 도중 부상 의심 교체 아웃...A매치...",https://m.sports.naver.com/wfootball/article/4...,20240928
61,"‘천만다행이다!’ 손흥민, 큰 부상 아니다···포스텍 감독 “SON, 훈련 참여 원...",https://m.sports.naver.com/wfootball/article/1...,20240928


In [57]:
# CSV 데이터 저장하기

import os

path = '/Users/ranny/GDSC EWHA/' # YOUR_PATH
file_name = f'해외축구_네이버_뉴스_{date_range[0]}_{date_range[-1]}.csv'

df.to_csv(os.path.join(path, file_name), index = False)